# Updating Hessians
The most common Hessian update formula is the BFGS update.  This formula results in a positive-definite Hessian matrix with all real eigenvalues, so it is unsuitable for transition-state optimizations.  For the formulae for BFGS, Powell, MS, and other update schemes see J.M. Bofill, _J. Comp. Chem._, 15, 1 (1994), or V. Bakken and T. Helgaker, _J. Chem. Phys._, 117, 9160 (2002).

Below, we will take a geometry optimization step, to obtain internal coordinate values and forces at two different points on the potential energy surface.  We will use the BFGS update to update the Hessian matrix.  

First, we create a molecule and guess an initial Hessian.

In [ ]:
import psi4
from psi4 import *
from psi4.core import *
import numpy as np
import os
sys.path.append('os.getcwd()')
from opt_helper import stre, bend, intcosMisc, linearAlgebra, displace

In [ ]:
mol = psi4.geometry("""
O
H 1 0.9
H 1 0.9 2 104
""")
# We'll use cc-pVDZ RHF.
psi4.set_options({"basis": "cc-pvdz"})
mol.update_geometry()

# Generate the internal coordinates manually.
intcos = [stre.STRE(0,1), stre.STRE(0,2), bend.BEND(1,0,2)]
for intco in intcos: 
    print(intco) 

# Handy variables for later.
Natom = mol.natom()
Nintco = len(intcos)
Ncart = 3*Natom
Z = [int(mol.Z(i)) for i in range(Natom)]

# Compute initial guess Hessian.
xyz = np.array(mol.geometry())

H = np.zeros((Nintco,Nintco), float)
for i,intco in enumerate(intcos):
    H[i,i] = intco.diagonalHessianGuess(xyz, Z, guessType="SCHLEGEL")

print("\n Schlegel Guess Hessian for Water (in au)")
print(H)

Next, we compute the internal coordinate values and forces at this point.  A separate tutorial shows how the transformation from Cartesians is accomplished.

In [ ]:
x_1 = xyz.copy()

# Compute internal coordinate values at current geometry.
q_1 = intcosMisc.qValues(intcos, x_1)

# Compute Cartesian gradient and transform to internals.
# The forces are simply -1 * the gradient.  The qForces function
# includes this sign change.
g_x1 = np.reshape( np.array( psi4.gradient('scf')), (3*Natom))
f_q1 = intcosMisc.qForces(intcos, xyz, g_x1)
E_1 = psi4.energy('scf')

We now take a simple Newton_Raphson step.  For more details on taking more complicated steps, see a separate tutorial.  After we take the step, we compute the internal coordinate forces at the new point.

In [ ]:
Hinv = linearAlgebra.symmMatInv(H, redundant=True)
dq = np.dot(Hinv, f_q1)
fq_aJ = intcosMisc.qShowForces(intcos, f_q1) # this is only used for printing.
displace.displace(intcos, xyz, dq, fq_aJ)

Compute forces at new coordinates.

In [ ]:
x_2 = xyz
q_2 = intcosMisc.qValues(intcos, x_2)

# Put new geometry into place and compute Cartesian gradient and internal forces.
mol.set_geometry( core.Matrix.from_array(x_2) )
mol.update_geometry()

g_x2 = np.reshape( np.array( psi4.gradient('scf')), (3*Natom))
f_q2 = intcosMisc.qForces(intcos, x_2, g_x2) # again, includes -1

OK, lets take a look at what we have in internal coordinates.

In [ ]:
print("Internal Values first step")
print(q_1)
print("Internal Values second step")
print(q_2)

print("Internal Forces first step")
print(f_q1)
print("Internal Forces second step")
print(f_q2)

We can see at a glance that the forces have been much reduced by the N-R step.  Also, that this step turned out to be somewhat too large, as the sign of all three forces in internal coordinates has changed.

Finally, we show how to use the gradients in internal coordinates to update the Hessian using the BFGS formula.  The vectors below should be read as rows.

   $$ \textbf{H}_{\rm{new}} = \textbf{H} + \frac{\delta g^T \delta g}{\delta q \delta g^T} - \frac{ \textbf{H} \delta q^T \big( \textbf{H} \delta q^T \big)^T }{ \delta q \textbf{H} \delta q^T} $$

In [ ]:
H_new = np.zeros( (Nintco,Nintco) , float)
dg = np.zeros(Nintco, float)

dq[:] = q_2 - q_1    # Change in internal values.
dg[:] = f_q1 - f_q2  # Change in gradient values (not forces).

gq = np.dot(dq, dg)  # A useful intermediate.

for i in range(Nintco):
    for j in range(Nintco):
        H_new[i,j] = H[i,j] + dg[i] * dg[j] / gq

Hdq = np.dot(H, dq)
dqHdq = np.dot(dq, Hdq)

for i in range(Nintco):
    for j in range(Nintco):
        H_new[i,j] -=  Hdq[i] * Hdq[j] / dqHdq
        
print("Updated Hessian with BFGS")
print(H_new)

We see that the Hessian in this case was not much changed by the update.  The first diagonal (stretching) element changes from 0.707 -> 0.714, while the bend changes from 0.16 -> 0.17.  However, if one repeats this demonstration with an initial Hessian guess set to "SIMPLE", then the initial Hessian guess is poorer (0.50 for the stretch), and the changes made by the first update (e.g., to 0.59 for the stretch) are more substantial.

Although many other Hessian update schemes have been proposed, for minimum-energy searches the BFGS method remains the method of choice.  In practice, there are some additional complications.  First, the denominators should be checked to see if they are too small.  They will become so if the change in the internal coordinate values or forces is very small (notice how the "gq" intermediate is computed).  In such cases, the update should not be performed.

The Hessian update may be carried out repeatedly using the current point and several (or all) past points in the optimization.  Using more than 1 previous point slightly improves average performance, but in some cases information from distant parts of the potential surface is not relevant or helpful.  This is certainly a parameter to try varying for problematic optimizations.